In [70]:
## Datas ###
#to_be_matched = [ "BATCH_REFERENCE", "TRANSACTION_REFERENCE","BUYER_NAME", "SUPPLIER_NAME" , "INVOICE_REFERENCE", "PO_REFERENCE",
#                    "INVOICE_DATE", "INVOICE_AMOUNT", "INVOICE_CURRENCY", "MATURITY_DATE", "PAYMENT_DATE", "TENOR", "NET_AMOUNT",
#                   "GROSS_AMOUNT", "DISCOUNT_AMOUNT", "ADJUSTMENT_AMOUNT", "ADJUSTMENT_REASON_CODE","PAYMENT_TERM", "STATUS", "NOTES"]

#to_be_matched = [ "Batch #", "Transaction #", "Buyer Name", "Supplier Name", "Inv #", "PO #", 
#"Invoice Date", "Invoice Amount", "Invoice Currency", "Maturity Date", "Payment Date", "Tenor", "Net Amount", "Gross Amount", "Discount Amount", "Adjustment Amount", "Adjustment Reason Code", "Payment Term", "Status", "Notes"]

to_be_matched = ["batchNum", "txnNum", "buyer", "supplier", "invNum", "poNum", "invDt", "invAmt", "invCcy", "matDt", "payDueDt", "tenor", "netAmt", "grossAmt", "discAmt", "adjustAmt", "adjustReason", "payTerm", "status", "notes"]

target = [ "batchReference", "transactionReference", "buyerName", 
            "supplierName", "invoiceReference", "poReference", 
            "invoiceDate", "invoiceAmount", "invoiceCurrency", "maturityDate", 
            "paymentDate", "tenor", "netAmount", "grossAmount", 
            "discountAmount", "adjustmentAmount", "adjustmentReasonCode", "paymentTerm", "status", "notes"]


In [71]:
import numpy as np
import pandas as pd
import re

In [72]:
dataset = pd.DataFrame({"to_be_matched":to_be_matched,"target":target})

In [73]:
dataset.head()

,to_be_matched,target
0,batchNum,batchReference
1,txnNum,transactionReference
2,buyer,buyerName
3,supplier,supplierName
4,invNum,invoiceReference


In [74]:
### Cleaning the to be matched column ###
def clean_strings(string):
    string = re.sub('[^a-zA-Z0-9.?]',' ',string)
    string = string.replace(' ','').lstrip().rstrip()
    return string.lower()

dataset["to_be_matched"].apply(clean_strings)

0         batchnum
1           txnnum
2            buyer
3         supplier
4           invnum
5            ponum
6            invdt
7           invamt
8           invccy
9            matdt
10        payduedt
11           tenor
12          netamt
13        grossamt
14         discamt
15       adjustamt
16    adjustreason
17         payterm
18          status
19           notes
Name: to_be_matched, dtype: object

In [75]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )

    return (matrix[size_x - 1, size_y - 1])

def levenshtein_rate(product1, product2):
    distance = levenshtein(product1, product2)
    max_len = max(len(product1), len(product2))
    return 1 - (distance / max_len)

score_list = [levenshtein_rate(list(dataset["to_be_matched"].apply(clean_strings))[i],list(dataset["target"].apply(clean_strings))[i]) for i in range(0,len(dataset["to_be_matched"]))]
#score_list = [levenshtein_rate(list(dataset["to_be_matched"])[i],list(dataset["target"])[i]) for i in range(0,len(dataset["to_be_matched"]))]
#round(sum(score_list)/len(score_list),2)
score_list

[0.4285714285714286,
 0.15000000000000002,
 0.5555555555555556,
 0.6666666666666667,
 0.25,
 0.2727272727272727,
 0.4545454545454546,
 0.46153846153846156,
 0.4,
 0.41666666666666663,
 0.4545454545454546,
 1.0,
 0.6666666666666667,
 0.7272727272727273,
 0.5,
 0.5625,
 0.6,
 0.6363636363636364,
 1.0,
 1.0]

In [76]:
from fuzzywuzzy import fuzz

fuzz_socre = [fuzz.token_sort_ratio(list(dataset["to_be_matched"].apply(clean_strings))[i],list(dataset["target"].apply(clean_strings))[i]) for i in range(0,len(dataset["to_be_matched"]))] 

fuzz_socre


C:\Users\eroynab\Documents\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[55,
 23,
 71,
 80,
 36,
 38,
 62,
 63,
 57,
 59,
 53,
 100,
 80,
 84,
 67,
 72,
 75,
 78,
 100,
 100]

In [77]:
fuzz_socre_abb = [fuzz.partial_token_sort_ratio(list(dataset["to_be_matched"].apply(clean_strings))[i],list(dataset["target"].apply(clean_strings))[i]) for i in range(0,len(dataset["to_be_matched"]))] 
fuzz_socre_abb

[62,
 33,
 100,
 100,
 50,
 40,
 60,
 50,
 67,
 60,
 50,
 100,
 83,
 88,
 57,
 78,
 67,
 57,
 100,
 100]

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

def ngrams(string, n=2):
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

#vectorizer = TfidfVectorizer(analyzer=ngrams)
vectorizer = pickle.load(open("vect.sav","rb"))

def cosine_sim(text1, text2, vect):
    #vectorizer = TfidfVectorizer(analyzer=ngrams)
    #vectorizer = pickle.load(open("vect.sav","rb"))
    tfidf = vect.transform([text1, text2])
    pickle.dump(vect,open("vect.sav","wb"))
    return ((tfidf * tfidf.T).A)[0,1]

In [47]:
print(ngrams(list(dataset["to_be_matched"].apply(clean_strings))[0]))

['ba', 'at', 'tc', 'ch', 'hr', 're', 'ef', 'fe', 'er', 're', 'en', 'nc', 'ce']


In [50]:
vect_score = [cosine_sim(list(dataset["to_be_matched"].apply(clean_strings)),list(dataset["target"].apply(clean_strings)),vectorizer) for i in range(0,len(dataset["to_be_matched"]))]

In [51]:
vect_score

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [27]:
vectorizer = pickle.load(open("vect.sav","rb"))

In [86]:
import spacy
! python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

doc1 = nlp(u'batchreference')
doc2 = nlp(u'BATCH_REFERENCE')

#print doc1.similarity(doc2)

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047114 sha256=7e74e92de3863eb24073cdbd1c2b80292ec02f9b3bba29c1dd1d334649ca174e
  Stored in directory: C:\Users\eroynab\AppData\Local\Temp\pip-ephem-wheel-cache-fbnmr_9f\wheels\b7\0d\f0\7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [91]:
import en_core_web_sm
nlp = en_core_web_sm.load()

doc1 = nlp(u'batchreference')
doc2 = nlp(u'BATCH_REFERENCE')

print (doc1.similarity(doc2))

0.2979752358504139


C:\Users\eroynab\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  import sys


## **Starting fresh**

In [116]:
import numpy as np
import pandas as pd
import re

In [138]:
## Make the data to train your model on ##
target = [ "batchReference", "transactionReference", "buyerName", 
            "supplierName", "invoiceReference", "poReference", 
            "invoiceDate", "invoiceAmount", "invoiceCurrency", "maturityDate", 
            "paymentDate", "tenor", "netAmount", "grossAmount", 
            "discountAmount", "adjustmentAmount", "adjustmentReasonCode", "paymentTerm", "status", "notes"]

source_1 = ["BATCH_REF", "TXN_REF", "BUYER", "SUPPLIER", "INV_REF", "PO_REF", "INV_DATE", "INV_AMT", "INV_CCY", "MAT_DATE", "PAY_DATE", "TENOR", "NET_AMT", "GROSS_AMT", "DISC_AMT", "ADJUST_AMT", "ADJUST_REASON_CODE", "PAY_TERM", "STATUS", "NOTE" ]
source_2 = ["batchNum", "txnNum", "buyer", "supplier", "invNum", "poNum", "invDt", "invAmt", "invCcy", "matDt", "payDueDt", "tenor", "netAmt", "grossAmt", "discAmt", "adjustAmt", "adjustReason", "payTerm", "status", "notes"]
source_3 = [ "BATCH_REFERENCE", "TRANSACTION_REFERENCE","BUYER_NAME", "SUPPLIER_NAME" , "INVOICE_REFERENCE", "PO_REFERENCE",
                    "INVOICE_DATE", "INVOICE_AMOUNT", "INVOICE_CURRENCY", "MATURITY_DATE", "PAYMENT_DATE", "TENOR", "NET_AMOUNT",
                   "GROSS_AMOUNT", "DISCOUNT_AMOUNT", "ADJUSTMENT_AMOUNT", "ADJUSTMENT_REASON_CODE","PAYMENT_TERM", "STATUS", "NOTES"]
source_4 = ["BatchNumber", "TransactionNumber", "Buyer", "Supplier", "InvoiceNumber", "PONumber", "InvoiceDate", "InvoiceAmount", "InvoiceCurrency", "MaturityDate", "PaymentDate", "Tenor", "NetAmt", "GrossAmt", "DiscountAmt", "AdjustmAmt", "AdjustReasonCode", "PayTerm", "Status", "Notes"]

In [139]:
### Create your dataset ###
### Ideal Mappings ###
dataset = pd.DataFrame({"source_format":source_1+source_2+source_3+source_4,"target_format":target+target+target+target})

In [144]:
shuffled_ordered_dataset = dataset.sample(frac=1)
shuffled_ordered_dataset

,source_format,target_format
23,supplier,supplierName
19,NOTE,notes
9,MAT_DATE,maturityDate
39,notes,notes
52,NET_AMOUNT,netAmount
...,...,...
75,AdjustmAmt,adjustmentAmount
65,PONumber,poReference
31,tenor,tenor
56,ADJUSTMENT_REASON_CODE,adjustmentReasonCode


In [147]:
#dataset = dataset.append(shuffled_ordered_dataset)
dataset.shape

(320, 2)

In [157]:
### Non-Ideal Mappings ###
from random import shuffle

source_final_list = source_1+source_2+source_3+source_4
target_final_list = target+target+target+target
shuffle(source_final_list)
#print(source_final_list)

#dataset["source_format"] = list(dataset["source_format"]) + source_final_list
#dataset["target_format"] = list(dataset["target_format"]) + target_final_list

#dataset = dataset.append(pd.DataFrame([list(source_final_list),list(target_final_list)],columns=dataset.columns))
#dataset["target_format"] = dataset["target_format"].append(pd.Series(target_final_list,index="target_format"))
shuffled_df = pd.DataFrame({"source_format":list(source_final_list),"target_format":list(target_final_list)})

In [158]:
dataset = dataset.append(shuffled_df)

In [159]:
dataset.shape

(640, 2)

In [160]:
### Getting the cleaned string for all ###
def clean_strings(string):
    string = re.sub('[^a-zA-Z0-9.?]',' ',string)
    string = string.replace(' ','').lstrip().rstrip()
    return string.lower()

dataset["source_format"] = dataset["source_format"].apply(clean_strings)
dataset["target_format"] = dataset["target_format"].apply(clean_strings)

In [161]:
score_list = [round(levenshtein_rate(list(dataset["source_format"])[i],list(dataset["target_format"])[i]),2) * 100 for i in range(0,len(dataset["source_format"]))]

dataset["lev_score"] = score_list

In [162]:
fuzz_lev_score = [fuzz.token_sort_ratio(list(dataset["source_format"])[i],list(dataset["target_format"])[i]) for i in range(0,len(dataset["source_format"]))]

dataset["fuzz_lev_score"] = fuzz_lev_score
dataset

,source_format,target_format,lev_score,fuzz_lev_score
0,batchref,batchreference,57.0,73
1,txnref,transactionreference,25.0,38
2,buyer,buyername,56.0,71
3,supplier,suppliername,67.0,80
4,invref,invoicereference,38.0,55
...,...,...,...,...
75,buyer,adjustmentamount,12.0,19
76,transactionnumber,adjustmentreasoncode,15.0,38
77,adjustmentreasoncode,paymentterm,25.0,39
78,adjustreasoncode,status,19.0,27


In [163]:
fuzz_abb_score = [fuzz.partial_token_sort_ratio(list(dataset["source_format"])[i],list(dataset["target_format"])[i]) for i in range(0,len(dataset["source_format"]))]

dataset["fuzz_abb_score"] = fuzz_abb_score
dataset

,source_format,target_format,lev_score,fuzz_lev_score,fuzz_abb_score
0,batchref,batchreference,57.0,73,100
1,txnref,transactionreference,25.0,38,67
2,buyer,buyername,56.0,71,100
3,supplier,suppliername,67.0,80,100
4,invref,invoicereference,38.0,55,67
...,...,...,...,...,...
75,buyer,adjustmentamount,12.0,19,20
76,transactionnumber,adjustmentreasoncode,15.0,38,50
77,adjustmentreasoncode,paymentterm,25.0,39,55
78,adjustreasoncode,status,19.0,27,67


In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

def ngrams(string, n=2):
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

#vectorizer = TfidfVectorizer(analyzer=ngrams)
vectorizer = pickle.load(open("vect.sav","rb"))

def cosine_sim(text1, text2, vect):
    #vectorizer = TfidfVectorizer(analyzer=ngrams)
    #vectorizer = pickle.load(open("vect.sav","rb"))
    tfidf = vect.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

vect_score = [round(cosine_sim(list(dataset["source_format"])[i],list(dataset["target_format"])[i],vectorizer),2)*100 for i in range(0,len(dataset["source_format"]))]

In [165]:
pickle.dump(vectorizer,open("vect.sav","wb"))

In [166]:
#vect_score 

In [167]:
dataset["vect_score"] = vect_score

In [168]:
dataset.shape

(640, 6)

In [169]:
dataset["match"] = [1 if i < 320 else 0 for i in range(0,dataset.shape[0])] 

In [171]:
dataset.to_csv("trainer.csv",index=False)

In [172]:
dataset

,source_format,target_format,lev_score,fuzz_lev_score,fuzz_abb_score,vect_score,match
0,batchref,batchreference,57.0,73,100,68.0,1
1,txnref,transactionreference,25.0,38,67,25.0,1
2,buyer,buyername,56.0,71,100,58.0,1
3,supplier,suppliername,67.0,80,100,69.0,1
4,invref,invoicereference,38.0,55,67,35.0,1
...,...,...,...,...,...,...,...
75,buyer,adjustmentamount,12.0,19,20,0.0,0
76,transactionnumber,adjustmentreasoncode,15.0,38,50,6.0,0
77,adjustmentreasoncode,paymentterm,25.0,39,55,12.0,0
78,adjustreasoncode,status,19.0,27,67,14.0,0


In [177]:
dataset.columns

Index(['source_format', 'target_format', 'lev_score', 'fuzz_lev_score',
       'fuzz_abb_score', 'vect_score', 'match'],
      dtype='object')

In [184]:
dataset.shape

(640, 7)

In [178]:
### SPlitting for train test set ####
from sklearn.model_selection import train_test_split

X = dataset[['lev_score', 'fuzz_lev_score','fuzz_abb_score', 'vect_score']]
y = dataset["match"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [181]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,f1_score,accuracy_score



dt_mod = DecisionTreeClassifier()
dt_mod.fit(X_train,y_train)

preds = dt_mod.predict(X_test)

print("############################################")
print(classification_report(y_test,preds))
print("############################################")
print(confusion_matrix(y_test,preds))
print("############################################")
print(accuracy_score(y_test,preds))
print("############################################")
print(f1_score(y_test,preds,average="weighted"))

############################################
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        97
           1       0.94      1.00      0.97       115

    accuracy                           0.97       212
   macro avg       0.97      0.96      0.97       212
weighted avg       0.97      0.97      0.97       212

############################################
[[ 90   7]
 [  0 115]]
############################################
0.9669811320754716
############################################
0.9668507522860634


In [201]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 20, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

mlp.fit(X_train,y_train)

mlp_preds = mlp.predict(X_test)

print("############################################")
print(classification_report(y_test,mlp_preds))
print("############################################")
print(confusion_matrix(y_test,mlp_preds))
print("############################################")
print(accuracy_score(y_test,mlp_preds))
print("############################################")
print(f1_score(y_test,mlp_preds,average="weighted"))

############################################
              precision    recall  f1-score   support

           0       0.94      0.92      0.93        97
           1       0.93      0.95      0.94       115

    accuracy                           0.93       212
   macro avg       0.93      0.93      0.93       212
weighted avg       0.93      0.93      0.93       212

############################################
[[ 89   8]
 [  6 109]]
############################################
0.9339622641509434
############################################
0.9339029630232053


In [198]:
mlp_preds = dt_mod.predict_proba(X_test)
print(mlp_preds[2][1]) ###2nd column
print(y_test)

0.9156626506024096
10    0
37    1
57    1
37    0
38    1
     ..
23    1
75    1
30    0
72    1
26    0
Name: match, Length: 212, dtype: int64


In [199]:
dt_mod.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [203]:
mlp.fit(X,y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 20, 20), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [200]:
###save dt mod ###
pickle.dump(dt_mod,open("decision_tree.sav","wb"))

In [204]:
pickle.dump(mlp,open("mlp.sav","wb"))